In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from dotenv import load_dotenv
import numpy as np
import json
import os
from typing import List, Dict
import geopandas as gpd
import plotly_express as px
import networkx as nx
from networkx.exception import NetworkXNoPath

load_dotenv()
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## TODO
1. Load Aleph entities
2. Load Mapstand data
3. Add missing geometries to mapstand, drop rows that are not used
4. Merge Mapstand with Aleph entities
5. Clean it up
6. Analyse country information and shapes
7. Analyse company information and shapes

In [ ]:
PATH_ALEPH = os.environ.get('PATH_WIND_ALEPH')
PATH_RAW = os.environ.get('PATH_RAW')
PATH_WIND = os.environ.get('PATH_WIND')

In [ ]:
def parse_json(entities: List) -> pd.DataFrame:
    '''Parses Aleph JSON data
    '''
    
    entity_list = []
    
    for entity in entities:
        data = entity.get('properties')

        for key, value in data.items():
            if isinstance(value, list):
                data.update({key: ','.join(value)})
        entity_id = {'id': entity.get('id')}
        data.update(entity_id)
        entity_list.append(data)
    
    df = pd.DataFrame(entity_list)
    return df


def load_entities(path: str, entity: str) -> pd.DataFrame:
    '''Load entities from Aleph
    (downloaded through alephclient)'''

    entities = []
    with open(f'{path}{entity}.json', 'r') as file:
        for line in file:
            entities.append(json.loads(line))

    df = parse_json(entities)
    return df

## Import Aleph entities

In [ ]:
# Import companies

companies = load_entities(PATH_ALEPH, 'companies')
companies.drop(['notes', 'summary', 'sourceUrl','publisher', 'alias', 'description', 
                'leiCode', 'parent', 'amountEur'], axis=1, inplace=True)


# Import and clean assets

assets = load_entities(PATH_ALEPH, 'assets')
assets.dropna(subset='description', inplace=True)
assets.drop(['title', 'authority', 'contractDate', 'jurisdiction', 'registrationNumber',
             'previousName', 'parent', 'leiCode', 'sourceUrl', 'publisher'], axis=1, inplace=True)

# Import ownerships

ownerships = load_entities(PATH_ALEPH, 'ownerships')

# Import legal entities

legalentities = load_entities(PATH_ALEPH, 'legalentities')
legalentities.drop(['idNumber', 'notes',
                    'publisherUrl', 'nationality', 'jurisdiction', 'previousName',
                    'registrationNumber', 'sourceUrl', 'summary', 'publisher', 'alias',
                    'description', 'leiCode', 'parent', 'amountEur'], axis=1, inplace=True)

# Import contracts
contracts = load_entities(PATH_ALEPH, 'contracts')

# Import contract awards

contractawards = load_entities(PATH_ALEPH, 'contractawards')

# Import people

persons = load_entities(PATH_ALEPH, 'persons')

# Import other links

other = load_entities(PATH_ALEPH, 'otherlinks')

## Import geometries


In [ ]:
# Import geojson

gdf = gpd.read_file(PATH_WIND + 'gis/wind_farms_20231012.geojson')

gdf = gdf.to_crs(28992)

# Drop columns

gdf.drop(['installed_capacity_mw', 'mps_est_shore_status', 'mps_est_area_sqkm', 'mps_id', 'capacity'], axis=1, inplace=True)

# Rename columns

cols = {'simple_status': 'status_simplified',
        'mps_id': 'mps_uuid'}

gdf = gdf.rename(columns=cols)

# There is some data missing

gdf.loc[gdf.mps_uuid=='f1328c2b-4821-4caf-b861-dfbd313c5ca2', 'year'] = 2028
gdf.loc[((gdf.country=='nl') & (gdf.status.isna())), 'status'] = 'EARLY_PLANNING'
gdf.loc[((gdf.country=='nl') & (gdf.status.isna())), 'status_simplified'] = 'EARLY_STAGE_PLANS'
gdf.loc[gdf.mps_uuid=='b1744eed-ee27-4b7a-be61-06fea1a3082e', 'status'] = 'CONSENT_AUTHORISED'
gdf.loc[gdf.mps_uuid=='b1744eed-ee27-4b7a-be61-06fea1a3082e', 'status_simplified'] = 'LATE_STAGE_PLANS'
gdf.loc[gdf.mps_uuid=='7581a6a1-a8e3-4d12-9f1c-79a89d73f685', 'status_simplified'] = 'EARLY_STAGE_PLANS'
gdf.loc[gdf.mps_uuid=='7581a6a1-a8e3-4d12-9f1c-79a89d73f685', 'status'] = 'EARLY_PLANNING'

# Clean dtypes

gdf['year'] = gdf.installation_year.apply(lambda x: pd.to_datetime(f"{str(x).replace('.0', '')}-01-01") if str(x) !='nan' else x)
gdf.drop('installation_year', axis=1, inplace=True)
cols = ['capacity_mw', 'value_eur', 'mps_est_elevation_min_m', 'mps_est_elevation_max_m', 'number_generators']
for col in cols:
        gdf[col] = np.floor(pd.to_numeric(gdf[col], errors='coerce')).astype('Int64')

# Create km2 area column

gdf['area_km'] = gdf.geometry.area / 1000000
gdf = gdf.to_crs(4326)
len(gdf)

In [ ]:
gdf.to_file(PATH_WIND + 'gis/windfarms_v1.geojson')

## Process ownership structure

There are several ways to go about this. It would be great if we could (partly) automate the generation of ownership tables, using graphs. We could use neo4j for that, or networkx. A query should look something like this:
1. For each asset assign a value of 1
2. Travel on an ownership relation and multiply the weight of that relationship, e.g. 1
3. At the next node find all ownership relationships
4. Travel all relationships and multiply by the weight of that relationship, e.g. .5


One of the problems with ownership is that we have some ranges and non-precise values (e.g. 75+). A solution is to define lower and upper values and convert them to weights. So this means we would create a few extra columns (percentage_lower_bound, percentage_upper_bound) and convert them to proper percentages so we can easily use them for multiplication. 

For now we have to assume that a missing percentage is 100 percent. That will often be te case, but we have to go through the Aleph data again one time to fill in the missing percentages.

One promising approach is to use Dijkstra's Algorithm, the shortest path, between companies (source) and assets (targets). Because we're dealing with a directed graph, this should omit any detours because companies have joint ventures in other projects. 

In [ ]:
# Clean percentages

ownerships.percentage = ownerships.percentage.str.replace('+', '-100')
ownerships.percentage.fillna('100', inplace=True)
ownerships.percentage = ownerships.percentage.astype('str')

# Add columns for lower and upper bound

ownerships['perc_lower'] = ownerships.percentage.apply(lambda x: float(x.split('-')[0]) / 100 if '-' in x else float(x) / 100)
ownerships['perc_higher'] = ownerships.percentage.apply(lambda x: float(x.split('-')[1]) / 100 if '-' in x else float(x) / 100)

In [ ]:
# Create directed graph

G = nx.DiGraph()

In [ ]:
for i, row in assets.iterrows():
    G.add_node(row.id,
               name=row['name'],
               status=row.notes,
               country=row.country, 
               costs=row.amountEur, 
               mps_uuid=row.description,
               aleph_url=row.alephUrl
               )
    
for i, row in companies.iterrows():
    G.add_node(row.id,
               name=row['name'],
               country=row.jurisdiction,
               registration=row.registrationNumber,
               source_url=row.publisherUrl,
               aleph_url=row.alephUrl
               )
    
for i, row in legalentities.iterrows():
    G.add_node(row.id,
               name=row['name'],
               country=row.country,
               aleph_url=row.alephUrl)
    
for i, row in ownerships.iterrows():
    G.add_edge(row.owner,
               row.asset,
               weight_lower=row.perc_lower,
               weight_upper=row.perc_higher,
               percentage=row.percentage,
               source=row.publisherUrl,
               aleph_url=row.alephUrl,
               id=id,
               description=row.description
               )
    




In [ ]:
# Get source, target and use Dijkstra's algorithm to get the nodes and relationships inbetween

def get_ownership(graph: nx.Graph, source: str, target: str, weight_range: str) -> Dict:
    '''Calculates ownership percentage
    Parameters:
    ----------
    graph: the graph with assets and owners (G)
    source: precise name of PSC/UBO
    target: precise name of asset
    weight_range: "lower" or "upper"
    '''
    
    # Get source and target node id
    try:
        s = [n for n, v in nx.get_node_attributes(graph, 'name').items() if v == source][0]
        t = [n for n, v in nx.get_node_attributes(graph, 'name').items() if v == target][0]
    except:
        return print(f'could not find entry for {source} and {target}')

    # Calculate shortest path
    try:
        [x for x in nx.dijkstra_path(G, s, t)]
        shortest_path = [x for x in nx.dijkstra_path(G, s, t)]
    except (KeyError, NetworkXNoPath) as error:
        #print(f'No path found between {source} and {target}')
        ownership = {}
        return ownership

    # Get node names for reference
    nodes = [x for x in shortest_path]
    names = [nx.get_node_attributes(G, 'name')[x] for x in nodes]

    # Traverse path and get edge weights
    window_size = 2
    weight = 1
    weights = []

    if weight_range == 'lower':
        weight_range = 'weight_lower'
    elif weight_range == 'upper':
        weight_range = 'weight_upper'
    else:
        raise ValueError('weight should be "lower" or "upper"')

    for i in range(len(shortest_path) - window_size + 1):
        node1 = shortest_path[i: i + window_size][0]
        node2 = shortest_path[i: i + window_size][1]
        nodes = list(G.edges([node1, node2], data=True))
        for node in nodes:
            if node[0] == node1 and node[1] == node2:
                e = node[2][weight_range] 
        weight *= e
        weights.append(e)

    # Create dict
    ownership = {'source_name': source,
                 'source_id': s,
                 'target_name': target,
                 'target_id': t,
                 'name_chain': " -> ".join(names),
                 'edge_weights': weights,
                 'ownership_percentage': weight}

    return ownership



In [ ]:
# Load list of ultimate beneficial owners

ubos = pd.read_csv(PATH_ALEPH + 'psc.csv')
ubos = ubos.company.to_list()

In [ ]:
bound = 'upper'

# Create a dataframe

o = []
for ubo in ubos:
    for i, row in assets.iterrows():
        ownership = get_ownership(G, ubo, row['name'], bound)
        o.append(ownership)

o = list(filter(None, o))
o_upper = pd.DataFrame(o)

In [ ]:
# Add capacity to it

df = pd.merge(o_upper,
              gdf[['name', 'mps_uuid', 'capacity_mw', 'year', 'number_generators', 'status_simplified', 'status', 'country', 'area_km']],
              left_on='target_name',
              right_on='name',
              how='left')

# Calculate capacity per company

df[f'output_company_mw_{bound}'] = df.ownership_percentage * df.capacity_mw
df[f'company_area_km_{bound}'] = df.ownership_percentage * df.area_km

# Rename columns

df = df.rename(columns={'source_name': 'company',
                         'target_name': 'asset',
                         'ownership_percentage': f'ownership_perc_{bound}'})

# Drop duplicates

df = df.drop_duplicates(subset=['company', 'asset']).copy()

In [ ]:
# Merge with company data

df_upper = pd.merge(df,
              companies[['name', 'jurisdiction']],
              left_on = 'company',
              right_on = 'name',
              how='left')


df_upper.drop(['name_x', 'name_y'], axis=1, inplace=True)

In [ ]:
df_upper = df_upper.rename(columns={'edge_weights': 'edge_weights_upper'})
df_lower = df_lower.rename(columns={'edge_weights': 'edge_weights_lower'})

In [ ]:
df_com = pd.merge(df_upper,
                 df_lower[['company', 'source_id', 'target_id', 'edge_weights_lower', 'output_company_mw_lower', f'company_area_km_lower', f'ownership_perc_lower']].copy(),
                 on=['company', 'source_id', 'target_id'],
                 how='left')



In [ ]:
df_com.head()

In [ ]:
df_com.to_csv(PATH_ALEPH + 'company_windfarm.csv', index=False)

In [ ]:
# Groupby company

coms = df_com.groupby('company').agg({'output_company_mw_lower': 'sum',
                                      'output_company_mw_upper': 'sum',
                                      'company_area_km_lower': 'sum',
                                      'company_area_km_upper': 'sum'})

coms['perc_output_lower'] = coms.output_company_mw_lower / coms.output_company_mw_lower.sum() * 100
coms['perc_output_upper'] = coms.output_company_mw_upper / coms.output_company_mw_upper.sum() * 100

coms['perc_area_lower'] = coms.company_area_km_lower / coms.company_area_km_lower.sum() * 100
coms['perc_area_upper'] = coms.company_area_km_upper / coms.company_area_km_upper.sum() * 100

coms.reset_index(inplace=True)

coms.to_csv(PATH_ALEPH + f'north_sea_company_output_lower_and_upper_bounds.csv', index=False)

In [ ]:
# Groupby company, country and status

country = df_com.groupby(['country', 'company', 'status', 'jurisdiction']).agg({'output_company_mw_lower': 'sum',
                                                                            'output_company_mw_upper': 'sum',
                                                                            'company_area_km_lower': 'sum',
                                                                            'company_area_km_upper': 'sum'})

country.reset_index(inplace=True)

country.to_csv(PATH_ALEPH + f'north_sea_total_output_grouped_by_country_and_company_and_status_lower_and_upper_bounds.csv', index=False)

## Contracts

In [ ]:
contracts.amount = contracts.amount.str.replace(' MW', '').apply(lambda x: str(x).split(' - ')[0]).astype('float')

In [ ]:
contracts = pd.merge(contracts,
                     contractawards[['contract', 'supplier']].copy(),
                     left_on = 'id', 
                     right_on = 'contract',
                     how='left')

len(contracts)

In [ ]:
contracts = pd.merge(contracts,
                     companies[['name', 'jurisdiction', 'id']].copy(),
                     left_on='supplier',
                     right_on='id',
                     how='left'
                     )

len(contracts)

In [ ]:
contracts.to_csv(PATH_ALEPH + 'ppa.csv', index=False)

In [ ]:
other = pd.merge(other,
                 assets,
                 left_on='subject',
                 right_on='id', 
                 how='left')

other

In [ ]:
# Check for endpoints (nodes without incoming edges)

endpoints = [node for node in G.nodes if G.in_degree(node) == 0]

for point in endpoints:
    node = G.nodes[point]
    if 'country' in node.keys():
        
        print(node['name'], node['country'])
    else:
        continue